---
---
# Scraping 
---
---

In [0]:
import sys   
from urllib.request import urlopen
from urllib.error import HTTPError, URLError
from bs4 import BeautifulSoup
import re
import pandas as pd

Dans un premier temps, nous avons récupéré les liens des pages thèmes de la FAQ de Maison du monde. Nous les avons sctockés dans un dictionnaire dico_liens qui contient en clé le nom du thème et en valeur son URL. 

In [2]:
dico_liens = {}
try:
    html = urlopen("https://faq-france.maisonsdumonde.com/fr/")
    bsObj = BeautifulSoup(html,"lxml")
    base = "https://faq-france.maisonsdumonde.com"
    soup = bsObj.find('div', class_='content educate_content')
    for link in soup.find_all("a"):
        h2 = link.find("h2",class_="t__h3 c__primary").get_text()
        dico_liens[h2] = base + link.get("href")
    
except (HTTPError, URLError) as e:
    sys.exit(e)

del dico_liens["Les questions les plus fréquentes"] #on les enlève car elles sont en double
dico_liens

{"Carte cadeau, avoir et bon d'achat": 'https://faq-france.maisonsdumonde.com/fr/collections/1359124-carte-cadeau-avoir-et-bon-d-achat',
 'Commande et Paiement': 'https://faq-france.maisonsdumonde.com/fr/collections/1235257-commande-et-paiement',
 'Livraison': 'https://faq-france.maisonsdumonde.com/fr/collections/1235252-livraison',
 'Mon Compte et Informations personnelles': 'https://faq-france.maisonsdumonde.com/fr/collections/1235277-mon-compte-et-informations-personnelles',
 'Produits et Disponibilité': 'https://faq-france.maisonsdumonde.com/fr/collections/1235242-produits-et-disponibilite',
 'Responsabilité Sociétale des Entreprises': 'https://faq-france.maisonsdumonde.com/fr/collections/1987503-responsabilite-societale-des-entreprises',
 'Retour et Remboursement': 'https://faq-france.maisonsdumonde.com/fr/collections/1235271-retour-et-remboursement'}

Dans un second temps, nous avons parcourus le dictionnaire dicos_liens afin de récupérer les URL de toutes les questions pour chaque thème. Nous les avons stockés dans un dictionnaire dico_liens_2 qui a pour clé un tuple composé de la question et du thème et comme valeur l'URL associée à la question.

In [3]:
dico_liens_2 = {}

for theme,url in dico_liens.items():
    try: 
        html = urlopen(url)
        bsObj = BeautifulSoup(html,"lxml")
        base = "https://faq-france.maisonsdumonde.com"
        soup = bsObj.find('div', class_='section__bg')
        for link in soup.find_all("a"):
            h2 = link.find("span",class_="c__primary").get_text()
            dico_liens_2[(h2,theme)] = base + link.get("href")
       
    
    except (HTTPError, URLError) as e:
        sys.exit(e)

dico_liens_2

{("Ab Custom : quelles sont les modalités d'une reprise ?",
  'Retour et Remboursement'): 'https://faq-france.maisonsdumonde.com/fr/articles/3349235-ab-custom-quelles-sont-les-modalites-d-une-reprise',
 ('Ab Custom : quels sont les délais et modalités d’une livraison ?',
  'Livraison'): 'https://faq-france.maisonsdumonde.com/fr/articles/3349233-ab-custom-quels-sont-les-delais-et-modalites-d-une-livraison',
 ('Agediss : quelles sont les modalités d’une reprise ?',
  'Retour et Remboursement'): 'https://faq-france.maisonsdumonde.com/fr/articles/2376588-agediss-quelles-sont-les-modalites-d-une-reprise',
 ('Agediss : quels sont les délais et modalités d’une livraison ?',
  'Livraison'): 'https://faq-france.maisonsdumonde.com/fr/articles/2362331-agediss-quels-sont-les-delais-et-modalites-d-une-livraison',
 ('Certains produits dégagent une odeur, faut-il s’en inquiéter ?',
  'Responsabilité Sociétale des Entreprises'): 'https://faq-france.maisonsdumonde.com/fr/articles/3524510-certains-produ

Enfin, nous effectuons un parcours de toutes les URL des questions afin de récupérer les réponses associées. Ainsi, le dictionnaire dico_liens_3 a pour clé l'intitulé de la question et son thème associé et pour valeur la réponse à la question. Nous avons également utilisé des expressions  régulières afin d'opérer un nettoyage des réponses obtenues. 

In [4]:
dico_liens_3 = {}

for cle,url in dico_liens_2.items():
    try: 
        html = urlopen(url)
        bsObj = BeautifulSoup(html,"lxml")
        soup_tags = bsObj.find('article').find_all(['p','ul'])
        list_text = [x.text for x in soup_tags]
        text = ' '.join(list_text)
        text = re.sub('\n','',text)
        text = re.sub('\xa0',' ',text)
        text = re.sub('\'',"'",text)
        text = re.sub('\s+',' ',text)
        dico_liens_3[cle] = text
        
    
    except (HTTPError, URLError) as e:
        sys.exit(e)

dico_liens_3

{("Ab Custom : quelles sont les modalités d'une reprise ?",
  'Retour et Remboursement'): 'La reprise est effectuée gratuitement chez vous, dans la pièce de votre choix. Pas la peine de remballer l’article, les livreurs s’occuperont de tout ! Il vous suffit juste de le démonter. Vous êtes contacté par téléphone pour convenir d’un rendez-vous du lundi au vendredi et de 9h à 18h (les plages horaires proposées peuvent varier selon votre code postal). Un créneau horaire de 2 heures vous est confirmé par SMS dès la prise de rendez-vous. A noter, en cas d’échange, la reprise et la relivraison ont lieu au même moment ! En cas de besoin, vous pouvez joindre Ab Custom au 0032 471 52 50 81 et leur communiquer votre numéro de bon de livraison (BL).',
 ('Ab Custom : quels sont les délais et modalités d’une livraison ?',
  'Livraison'): "Vous êtes livré par Ab Custom sous 6 à 8 jours ouvrés et dans la pièce de votre choix. Une fois la commande remise au transporteur, vous recevez un e-mail de confi

Nous avons créé un dataframe pandas contenant les questions, les réponses et le thème associé. Nous l'avons ensuite exporté en CSV et c'est ce qui va nous servir de corpus orienté tâche pour la réalisation du chatbot. 


In [7]:
liste = [] #liste de dicos
for question, reponse in dico_liens_3.items():
    dico = {}
    dico["theme"] = question[1]
    dico["question"] = question[0]
    dico["reponse"] = reponse
    liste.append(dico)

df = pd.DataFrame(liste)
#df.to_csv("scraping.csv",index=False, sep=";")
df.groupby("theme").count()

,question,reponse
theme,,
"Carte cadeau, avoir et bon d'achat",19,19
Commande et Paiement,14,14
Livraison,31,31
Mon Compte et Informations personnelles,10,10
Produits et Disponibilité,20,20
Responsabilité Sociétale des Entreprises,22,22
Retour et Remboursement,26,26
